このColabのライセンスはApache License 2.0

引用元

https://note.com/kohya_ss/n/nee3ed1649fb6

https://note.com/kohya_ss/n/n2693183a798e

In [ ]:
#@title GPUチェック
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [ ]:
#@title パッケージのインストール
!pip install -q diffusers[torch]==0.9.0 accelerate transformers==4.21.3 ftfy albumentations opencv-python einops bitsandbytes fairscale==0.4.6 numpy==1.21.6

!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.14/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl

!wget -q https://github.com/thx-pw/stable-diffusion-2.0-dreambooth/raw/main/gen_img_diffusers.py
!wget -q https://github.com/thx-pw/stable-diffusion-2.0-dreambooth/raw/main/train_db_fixed.py

In [ ]:
#@title 768サイズのSD2.0モデルをダウンロード
!wget https://huggingface.co/stabilityai/stable-diffusion-2/resolve/main/768-v-ema.ckpt -P /content

In [ ]:
#@title 教師データのアップロードと正則化画像の自動生成
#@markdown このセルでSKSとCLASSを変更し、複数回実行すると複数キャラの学習が可能

#@markdown Colabへのアップロードが遅いのでリサイズするツール：https://www.birme.net/?target_width=768&target_height=768
SKS = "zunko" #@param {type:"string"}
CLASS = "girl" #@param {type:"string"}
TRAIN_N_REPEATS = 20 #@param {type:"integer"}
REG_N_REPEATS = 1 #@param {type:"integer"}

import os
from google.colab import files
import shutil
import glob
from PIL import Image

# 教師データ（学習データ）の保存場所
TRAIN_DIR = "/content/input/train"
!mkdir -p $TRAIN_DIR

# 正則化画像（クラスの画像）の保存場所
REG_DIR = "/content/input/reg"
!mkdir -p $REG_DIR

# 学習済みモデルの保存場所
OUTPUT_DIR = "/content/output" 
!mkdir -p $OUTPUT_DIR

train_path = os.path.join(TRAIN_DIR, f"{TRAIN_N_REPEATS}_{SKS} {CLASS}")
os.makedirs(train_path, exist_ok=True)
reg_path = os.path.join(REG_DIR, f"{REG_N_REPEATS}_{CLASS}")
os.makedirs(reg_path, exist_ok=True)

uploaded = files.upload()
for filename in uploaded.keys():
    dst_path = os.path.join(train_path, filename)
    shutil.move(filename, dst_path)


train_num_images = sum(os.path.isfile(os.path.join(train_path, name)) for name in os.listdir(train_path))
if train_num_images > 0:
  reg_num_images = sum(os.path.isfile(os.path.join(reg_path, name)) for name in os.listdir(reg_path))
  reg_num_images = (TRAIN_N_REPEATS * train_num_images) // REG_N_REPEATS - reg_num_images
  !python gen_img_diffusers.py \
    --ckpt 768-v-ema.ckpt \
    --outdir {reg_path} \
    --xformers \
    --fp16 \
    --v2 --v_parameterization \
    --W 768 \
    --H 768 \
    --scale 12.5 \
    --sampler ddim \
    --steps 20 \
    --batch_size 4 \
    --images_per_prompt {reg_num_images} \
    --prompt "{CLASS} -n"
else:
  print("cancel upload.")


In [ ]:
#@title DreamBoothで学習開始
!accelerate launch --num_cpu_threads_per_process 2 train_db_fixed.py \
  --pretrained_model_name_or_path="/content/768-v-ema.ckpt" \
  --train_data_dir=$TRAIN_DIR \
  --reg_data_dir=$REG_DIR \
  --output_dir=$OUTPUT_DIR \
  --prior_loss_weight=1.0 \
  --resolution=768 \
  --train_batch_size=4 \
  --learning_rate=1e-6 \
  --max_train_steps=400 \
  --use_8bit_adam \
  --mixed_precision='fp16' \
  --xformers \
  --cache_latents \
  --gradient_checkpointing \
  --v2 \
  --v_parameterization \
  --save_precision='fp16' \
  --save_every_n_epochs 2 \
  --logging_dir=logs

In [ ]:
#@title ログの確認
%load_ext tensorboard
%tensorboard --logdir=logs

In [ ]:
#@title Google Driveにckptを保存
ckpt_name = "last" #@param {type:"string"}
from google.colab import drive
drive.mount('/content/drive')

import os
model_checkpoints = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion"
os.makedirs(model_checkpoints, exist_ok=True)
!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O "{OUTPUT_DIR}/{ckpt_name}.yaml"
!cp "{OUTPUT_DIR}/{ckpt_name}.ckpt" {model_checkpoints}
!cp "{OUTPUT_DIR}/{ckpt_name}.yaml" {model_checkpoints}

print(f"save to {model_checkpoints}")

In [ ]:
#@title 学習済みモデルで画像生成
!python gen_img_diffusers.py \
  --ckpt "{OUTPUT_DIR}/{ckpt_name}.ckpt" \
  --outdir 'tmp' \
  --xformers \
  --fp16 \
  --v2 --v_parameterization \
  --W 768 \
  --H 768 \
  --scale 12.5 \
  --sampler ddim \
  --steps 20 \
  --batch_size 4 \
  --images_per_prompt 4 \
  --prompt "{SKS} {CLASS} eating a lunch in MacDonald's -n"

print("create to /content/tmp")